In [ ]:
# ============================================================================
# CRITICAL FIX: Must be executed FIRST
# ============================================================================
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

print("Environment variables set successfully")
print("="*60)

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

seed = 42
np.random.seed(seed)

print("="*60)
print("IMPORTING TENSORFLOW, KERAS AND TRANSFORMERS...")
print("="*60)

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling1D, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)

# NUEVO: Transformers library
from transformers import TFDistilBertModel, DistilBertTokenizer
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)

print("="*60)
print("ENVIRONMENT SETUP - SUCCESS")
print("="*60)
print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers available: True")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
if tf.config.list_physical_devices('GPU'):
    gpu_devices = tf.config.list_physical_devices('GPU')
    print(f"GPU Devices: {len(gpu_devices)} device(s)")
    for gpu in gpu_devices:
        print(f"  - {gpu.name}")
print(f"Random seed: {seed}")
print("="*60)

## Iteración 4 - TRANSFER LEARNING CON DISTILBERT

**Evolución Arquitecturas:**
- V1-V3: LSTM Bidireccional (embeddings entrenables desde cero)
- V4: DistilBERT (66M parámetros pre-entrenados)

**Ventajas DistilBERT:**
1. Embeddings contextuales (entiende significado según contexto)
2. Pre-entrenado en millones de textos
3. Captura relaciones complejas entre palabras
4. Mejor comprensión semántica para SPAM
5. 40% más rápido que BERT completo

**Estrategia Fine-tuning:**
- Congelar capas base (primeras 4/6)
- Fine-tune últimas 2 capas
- Clasificador custom con dropout
- Learning rate bajo (2e-5)
- Gradient accumulation para batch efectivo mayor

**Objetivo:** MCC > 0.90 (salto significativo vs 0.8885 de V2)

In [ ]:
# Hyperparameters Configuration - ITERACIÓN 4 - TRANSFER LEARNING

# DistilBERT Configuration
MODEL_NAME = 'distilbert-base-uncased'  # Modelo pre-entrenado
MAX_LEN = 128                           # BERT optimal length (vs 200 de LSTM)
BATCH_SIZE = 16                         # Reducido por memoria (vs 32)
GRADIENT_ACCUMULATION_STEPS = 2         # Batch efectivo = 16*2 = 32

# Fine-tuning Configuration
LEARNING_RATE = 2e-5                    # Standard fine-tuning LR
WARMUP_STEPS = 100                      # Warmup para estabilidad
EPOCHS = 10                             # Menos epochs que LSTM (converge rápido)
VALIDATION_SPLIT = 0.2

# Classifier Configuration
CLASSIFIER_DROPOUT = 0.3                # Dropout en clasificador custom
DENSE_UNITS = 128                       # Unidades capa intermedia
L2_REG = 1e-4                          # Regularización L2

# Layer Freezing Strategy
FREEZE_BASE = True                      # Congelar base inicialmente
UNFREEZE_LAST_N_LAYERS = 2             # Descongelar últimas 2 capas

print("="*60)
print("MODEL CONFIGURATION - ITERACIÓN 4")
print("TRANSFER LEARNING - DISTILBERT")
print("="*60)
print("CAMBIO ARQUITECTURAL RADICAL")
print("="*60)
print(f"Base Model: {MODEL_NAME}")
print(f"  - Pre-trained parameters: ~66M")
print(f"  - Transformer layers: 6")
print(f"  - Attention heads: 12")
print(f"  - Hidden size: 768")
print("="*60)
print("Configuration:")
print(f"  Max Length: {MAX_LEN} (optimal for BERT)")
print(f"  Batch Size: {BATCH_SIZE} (physical)")
print(f"  Gradient Accumulation: {GRADIENT_ACCUMULATION_STEPS} steps")
print(f"  Effective Batch Size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  Learning Rate: {LEARNING_RATE} (fine-tuning)")
print(f"  Warmup Steps: {WARMUP_STEPS}")
print(f"  Max Epochs: {EPOCHS}")
print(f"  Classifier Dropout: {CLASSIFIER_DROPOUT}")
print(f"  Dense Units: {DENSE_UNITS}")
print("="*60)
print("Fine-tuning Strategy:")
print(f"  Freeze base: {FREEZE_BASE}")
print(f"  Unfreeze last N layers: {UNFREEZE_LAST_N_LAYERS}")
print(f"  Total trainable: Classifier + Last {UNFREEZE_LAST_N_LAYERS} transformer layers")
print("="*60)
print("\nEsperado:")
print("  - MCC > 0.90 (vs 0.8885 de V2)")
print("  - Mejor comprensión semántica")
print("  - Menos overfitting (pre-training robusto)")
print("  - Tiempo: ~5-8 min (transformers más lentos)")
print("="*60)

## Carga y Exploración de Datos

In [ ]:
# Load training data
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")

print("="*60)
print("TRAINING DATA OVERVIEW")
print("="*60)
print(f"Total samples: {len(train):,}")
print(f"\nClass distribution:\n{train['spam_label'].value_counts()}")
print(f"\nClass balance:\n{train['spam_label'].value_counts(normalize=True)}")
print("="*60)

train.head(10)

## Tokenización con DistilBERT

**Diferencias vs Custom Tokenizer:**
- WordPiece tokenization (subpalabras)
- Vocabulario pre-entrenado 30,522 tokens
- Tokens especiales: [CLS], [SEP], [PAD]
- Attention masks para padding
- Compatible con embeddings DistilBERT

In [ ]:
# Inicializar tokenizer DistilBERT
print("Cargando tokenizer DistilBERT...")
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

print("="*60)
print("DISTILBERT TOKENIZER LOADED")
print("="*60)
print(f"Vocabulary size: {tokenizer.vocab_size:,}")
print(f"Max length: {MAX_LEN}")
print(f"Special tokens: {tokenizer.special_tokens_map}")
print("="*60)

# Ejemplo de tokenización
sample_text = train['text'].iloc[0]
print(f"\nEjemplo tokenización:")
print(f"Original: {sample_text[:100]}...")
encoded = tokenizer.encode_plus(
    sample_text,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='tf'
)
print(f"Tokens: {encoded['input_ids'].shape}")
print(f"Attention mask: {encoded['attention_mask'].shape}")
print(f"\nPrimeros 10 tokens: {tokenizer.convert_ids_to_tokens(encoded['input_ids'][0][:10])}")

In [ ]:
# Tokenizar dataset completo
def tokenize_texts(texts, tokenizer, max_len):
    """
    Tokeniza lista de textos usando DistilBERT tokenizer
    Retorna input_ids y attention_masks
    """
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids), np.array(attention_masks)

print("Tokenizando textos con DistilBERT...")
X_train_text = train['text'].values
y_train = train['spam_label'].values

X_train_ids, X_train_masks = tokenize_texts(X_train_text, tokenizer, MAX_LEN)

# Split train/validation
X_train_ids_final, X_val_ids, X_train_masks_final, X_val_masks, y_train_final, y_val = train_test_split(
    X_train_ids, X_train_masks, y_train,
    test_size=VALIDATION_SPLIT,
    random_state=seed,
    stratify=y_train
)

print("="*60)
print("TOKENIZATION SUMMARY")
print("="*60)
print(f"Training samples: {len(X_train_ids_final):,}")
print(f"Validation samples: {len(X_val_ids):,}")
print(f"Input shape: {X_train_ids_final.shape}")
print(f"Attention mask shape: {X_train_masks_final.shape}")
print(f"Train class distribution: {np.bincount(y_train_final)}")
print(f"Val class distribution: {np.bincount(y_val)}")
print("="*60)

## Construcción del Modelo - DistilBERT + Custom Classifier

**Arquitectura:**
```
Input (text) → DistilBERT Tokenizer → [CLS] token
    ↓
DistilBERT Base (6 transformer layers)
    ↓ (freeze first 4 layers)
Trainable Transformer Layers (last 2)
    ↓
Global Average Pooling
    ↓
Dense(128, relu) + L2 + Dropout(0.3)
    ↓
Dense(1, sigmoid) → SPAM probability
```

**Fine-tuning Strategy:**
- Fase 1: Solo entrenar clasificador (2-3 epochs)
- Fase 2: Descongelar últimas capas (resto de epochs)

In [ ]:
# Crear modelo custom usando Model Subclassing
class DistilBertSpamClassifier(keras.Model):
    def __init__(self, model_name, dense_units, dropout_rate, l2_reg, freeze_base=True):
        super(DistilBertSpamClassifier, self).__init__()
        
        # Cargar DistilBERT pre-entrenado
        self.distilbert = TFDistilBertModel.from_pretrained(model_name)
        
        # Congelar si es necesario
        if freeze_base:
            self.distilbert.trainable = False
            print("Base DistilBERT congelado. Se entrenará solo el clasificador.")
        
        # Capas del clasificador
        self.pooling = GlobalAveragePooling1D()
        self.dense1 = Dense(
            dense_units,
            activation='relu',
            kernel_regularizer=l2(l2_reg),
            name='dense_1'
        )
        self.dropout = Dropout(dropout_rate)
        self.output_layer = Dense(1, activation='sigmoid', name='output')
    
    def call(self, inputs, training=False):
        input_ids, attention_mask = inputs
        
        # Forward pass DistilBERT
        distilbert_output = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            training=training
        )
        
        # Usar hidden states
        sequence_output = distilbert_output.last_hidden_state
        
        # Pooling
        pooled = self.pooling(sequence_output)
        
        # Clasificador
        x = self.dense1(pooled)
        x = self.dropout(x, training=training)
        output = self.output_layer(x)
        
        return output

# Construir modelo
print("Construyendo modelo DistilBERT...")
model = DistilBertSpamClassifier(
    model_name=MODEL_NAME,
    dense_units=DENSE_UNITS,
    dropout_rate=CLASSIFIER_DROPOUT,
    l2_reg=L2_REG,
    freeze_base=FREEZE_BASE
)

# Build model con inputs dummy para inicializar pesos
dummy_input_ids = tf.zeros((1, MAX_LEN), dtype=tf.int32)
dummy_attention_mask = tf.zeros((1, MAX_LEN), dtype=tf.int32)
_ = model([dummy_input_ids, dummy_attention_mask], training=False)

print("Modelo construido exitosamente.")

# Compilar
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

# Resumen
model.summary()

print("\n" + "="*60)
print("DISTILBERT MODEL COMPILED")
print("="*60)
print(f"Total parameters: {model.count_params():,}")

# Contar trainable params
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable_params = model.count_params() - trainable_params

print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {non_trainable_params:,}")
print(f"\nComparación con versiones anteriores:")
print(f"  V1 LSTM: 1,251,009 params (100% trainable)")
print(f"  V2 LSTM: 1,160,609 params (100% trainable)")
print(f"  V3 LSTM: ~1,000,000 params (100% trainable)")
print(f"  V4 DistilBERT: {model.count_params():,} params ({trainable_params:,} trainable)")
print(f"\nOptimizer: Adam (lr={LEARNING_RATE})")
print(f"Loss: Binary Crossentropy")
print("="*60)

# Guardar referencia para fine-tuning
distilbert_layer = model.distilbert

## Entrenamiento - Fase 1: Solo Clasificador

**Estrategia 2 fases:**
1. Entrenar solo clasificador (base congelado) - 3 epochs
2. Descongelar últimas capas y fine-tune - resto epochs

Esta estrategia evita catastrofic forgetting del pre-training.

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_distilbert_spam_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]

print("="*60)
print("FASE 1: ENTRENAMIENTO CLASIFICADOR")
print("="*60)
print("Base DistilBERT: CONGELADO")
print("Clasificador: ENTRENABLE")
print("Epochs: 3")
print("="*60)

# Fase 1: Solo clasificador (3 epochs)
history_phase1 = model.fit(
    [X_train_ids_final, X_train_masks_final],
    y_train_final,
    batch_size=BATCH_SIZE,
    epochs=3,
    validation_data=([X_val_ids, X_val_masks], y_val),
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("FASE 1 COMPLETADA")
print("="*60)

## Entrenamiento - Fase 2: Fine-tuning Últimas Capas

In [ ]:
# Descongelar últimas N capas de DistilBERT
print("="*60)
print("FASE 2: FINE-TUNING ÚLTIMAS CAPAS")
print("="*60)

if FREEZE_BASE:
    # Descongelar todo el modelo DistilBERT
    distilbert_layer.trainable = True
    
    # Acceder a las capas transformer internas
    # DistilBERT tiene 6 capas transformer en distilbert.transformer.layer
    transformer_layers = distilbert_layer.distilbert.transformer.layer
    total_transformer_layers = len(transformer_layers)
    
    # Congelar todas las capas transformer excepto las últimas N
    layers_to_freeze = max(0, total_transformer_layers - UNFREEZE_LAST_N_LAYERS)
    
    # Congelar embeddings
    distilbert_layer.distilbert.embeddings.trainable = False
    
    # Congelar capas transformer según estrategia
    for i, layer in enumerate(transformer_layers):
        if i < layers_to_freeze:
            layer.trainable = False
        else:
            layer.trainable = True
    
    print(f"Total transformer layers: {total_transformer_layers}")
    print(f"Transformer layers congeladas: {layers_to_freeze}")
    print(f"Transformer layers entrenables: {UNFREEZE_LAST_N_LAYERS}")
    print(f"Embeddings: CONGELADOS")
    
    # Re-compilar con learning rate más bajo
    optimizer_phase2 = keras.optimizers.Adam(learning_rate=LEARNING_RATE / 10)
    
    model.compile(
        optimizer=optimizer_phase2,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )
    
    trainable_params_phase2 = sum([tf.size(w).numpy() for w in model.trainable_weights])
    print(f"\nTrainable parameters fase 2: {trainable_params_phase2:,}")
    print(f"Learning rate: {LEARNING_RATE / 10}")
    print("="*60)
    
    # Fase 2: Fine-tuning (resto de epochs)
    history_phase2 = model.fit(
        [X_train_ids_final, X_train_masks_final],
        y_train_final,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        initial_epoch=3,
        validation_data=([X_val_ids, X_val_masks], y_val),
        callbacks=callbacks,
        verbose=1
    )
    
    print("\n" + "="*60)
    print("FASE 2 COMPLETADA")
    print("="*60)
    
    # Combinar historiales
    history = history_phase1
    for key in history_phase2.history:
        history.history[key].extend(history_phase2.history[key])
else:
    history = history_phase1

print("\nENTRENAMIENTO COMPLETO")

## Evaluación del Modelo

In [ ]:
# Predicciones en validación
y_pred_proba = model.predict([X_val_ids, X_val_masks], batch_size=BATCH_SIZE)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

mcc_score = matthews_corrcoef(y_val, y_pred)

print("="*60)
print("VALIDATION METRICS - DISTILBERT")
print("="*60)
print(f"Matthews Correlation Coefficient: {mcc_score:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=['Not SPAM', 'SPAM']))
print("="*60)

# Matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Not SPAM', 'SPAM'],
            yticklabels=['Not SPAM', 'SPAM'])
plt.title(f'DistilBERT Confusion Matrix (MCC: {mcc_score:.4f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## Análisis Comparativo - LSTM vs DistilBERT

In [ ]:
# Análisis comparativo
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
overfitting_delta = abs(final_val_loss - final_train_loss)

print("\n" + "="*80)
print("ANÁLISIS COMPARATIVO - LSTM vs DISTILBERT")
print("="*80)

comparison_data = {
    'V1 LSTM': {'mcc': 0.8665, 'arch': 'Bi-LSTM(128)', 'params': '1.25M'},
    'V2 LSTM': {'mcc': 0.8885, 'arch': 'Bi-LSTM(96)+L2', 'params': '1.16M'},
    'V3 LSTM': {'mcc': 0.8733, 'arch': 'Bi-LSTM(64)+L2x5', 'params': '~1.0M'},
    'V4 DistilBERT': {'mcc': mcc_score, 'arch': 'DistilBERT+FT', 'params': f'{model.count_params()/1e6:.1f}M'}
}

print("\nMCC Evolution:")
for version, data in comparison_data.items():
    print(f"  {version}: {data['mcc']:.4f} | {data['arch']} | {data['params']} params")

best_lstm = 0.8885  # V2
improvement = mcc_score - best_lstm

print("\n" + "="*80)
print("RESULTADO TRANSFER LEARNING:")
print("="*80)
print(f"MCC DistilBERT: {mcc_score:.4f}")
print(f"Best LSTM (V2): {best_lstm:.4f}")
print(f"Mejora: {'+' if improvement > 0 else ''}{improvement:.4f} ({improvement/best_lstm*100:+.1f}%)")
print(f"\nOverfitting Delta: {overfitting_delta:.4f}")
print(f"Val Accuracy: {final_val_acc:.4f}")

if mcc_score > 0.90:
    print("\nOBJETIVO CUMPLIDO: MCC > 0.90")
elif mcc_score > best_lstm:
    print("\nMEJORA CONFIRMADA: Transfer learning superior a LSTM")
elif mcc_score > best_lstm - 0.01:
    print("\nRESULTADO SIMILAR: Transfer learning competitivo")
else:
    print("\nATENCIÓN: LSTM V2 sigue siendo mejor")
    print("Posibles causas: fine-tuning insuficiente, hyperparams subóptimos")

print("="*80)

## Predicciones en Test Data

In [ ]:
# Load test data
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")

print("="*60)
print("TEST DATA")
print("="*60)
print(f"Total test samples: {len(test):,}")
print("="*60)

# Tokenizar test data
print("\nTokenizando test data con DistilBERT...")
X_test_text = test['text'].values
X_test_ids, X_test_masks = tokenize_texts(X_test_text, tokenizer, MAX_LEN)

print(f"Test shape: {X_test_ids.shape}")

# Generar predicciones
print("\nGenerando predicciones...")
y_pred_proba_test = model.predict([X_test_ids, X_test_masks], batch_size=BATCH_SIZE)
y_pred_test = (y_pred_proba_test > 0.5).astype(int).flatten()

print(f"\nPredicciones generadas: {len(y_pred_test):,}")
print(f"Distribución:")
print(f"  Not SPAM: {np.sum(y_pred_test == 0):,} ({np.mean(y_pred_test == 0)*100:.2f}%)")
print(f"  SPAM: {np.sum(y_pred_test == 1):,} ({np.mean(y_pred_test == 1)*100:.2f}%)")

## Generación de Submission

In [ ]:
# Crear submission file
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred_test
submission.to_csv('submission.csv', index=False)

print("="*60)
print("SUBMISSION FILE CREATED - DISTILBERT V4")
print("="*60)
print(f"Total predictions: {len(submission):,}")
print(f"File: submission.csv")
print("="*60)

submission.head(10)

## Resumen Final

In [ ]:
# Tabla resumen final
summary_df = pd.DataFrame({
    'Iteración': ['V1', 'V2', 'V3', 'V4'],
    'Arquitectura': ['Bi-LSTM(128)', 'Bi-LSTM(96)+L2', 'Bi-LSTM(64)+L2x5', 'DistilBERT+FT'],
    'Val MCC': [0.8665, 0.8885, 0.8733, f'{mcc_score:.4f}'],
    'Parámetros': ['1.25M', '1.16M', '~1.0M', f'{model.count_params()/1e6:.1f}M'],
    'Trainable': ['1.25M', '1.16M', '~1.0M', f'{trainable_params/1e6:.1f}M'],
    'Approach': ['Baseline', 'Regularization', 'Shock Therapy', 'Transfer Learning']
})

print("="*90)
print("RESUMEN FINAL - TODAS LAS ITERACIONES")
print("="*90)
print(summary_df.to_string(index=False))
print("="*90)

print("\nCONCLUSIONES:")
print(f"  - Transfer Learning MCC: {mcc_score:.4f}")
print(f"  - Mejor LSTM (V2): 0.8885")
print(f"  - Diferencia: {mcc_score - 0.8885:.4f}")
print("\nVENTAJAS DISTILBERT:")
print("  + Embeddings contextuales pre-entrenados")
print("  + Comprensión semántica superior")
print("  + Menor overfitting (pre-training robusto)")
print("\nDESVENTAJAS:")
print("  - Mayor consumo memoria")
print("  - Entrenamiento más lento")
print("  - Requiere fine-tuning cuidadoso")
print("="*90)